In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/harmful-speech/CT22_arabic_1C_harmful_dev.tsv
/kaggle/input/harmful-speech/CT22_arabic_1C_harmful_dev_test.tsv
/kaggle/input/harmful-speech/CT22_arabic_1C_harmful_train.tsv
/kaggle/input/checkworthiness/checkworthiness/CT22_arabic_1A_checkworthy_dev_test.tsv
/kaggle/input/checkworthiness/checkworthiness/CT22_arabic_1A_checkworthy_train.tsv
/kaggle/input/checkworthiness/checkworthiness/CT22_arabic_1A_checkworthy_dev.tsv


In [2]:
train=pd.read_csv('../input/checkworthiness/checkworthiness/CT22_arabic_1A_checkworthy_train.tsv',sep='\t')
test=pd.read_csv('../input/checkworthiness/checkworthiness/CT22_arabic_1A_checkworthy_dev_test.tsv',sep='\t')
dev=pd.read_csv('../input/checkworthiness/checkworthiness/CT22_arabic_1A_checkworthy_dev.tsv',sep='\t')

In [3]:
def remove_(line):
    return re.sub('[!@#$?؟]', '', line)
    

In [4]:
import re

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [5]:
train['tweet_text'] = train['tweet_text'].apply(lambda x:remove_emoji(x))
test['tweet_text'] = test['tweet_text'].apply(lambda x:remove_emoji(x))
dev['tweet_text'] = dev['tweet_text'].apply(lambda x:remove_emoji(x))
train['tweet_text'] = train['tweet_text'].apply(lambda x:remove_(x))
test['tweet_text'] = test['tweet_text'].apply(lambda x:remove_(x))
dev['tweet_text'] = dev['tweet_text'].apply(lambda x:remove_(x))

In [6]:
!pip install nlpaug

     |████████████████████████████████| 410 kB 253 kB/s            


In [7]:
import nlpaug.augmenter.word as naw

In [8]:
aug = naw.ContextualWordEmbsAug(
    model_path='aubmindlab/bert-base-arabertv01', action="substitute")


Downloading:   0%|          | 0.00/379 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/762k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

In [9]:
df=pd.DataFrame(columns=['topic','tweet_id','tweet_text','class_label'])
for i in range(len(train)):
    if(train['class_label'][i]==1):
        dict = {'topic':train['topic'][i] , 'tweet_id':train['tweet_id'][i], 'tweet_text':aug.augment(train['tweet_text'][i]), 'class_label':train['class_label'][i] }
        df = df.append(dict, ignore_index = True)

In [10]:
df2=pd.DataFrame(columns=['topic','tweet_id','tweet_text','class_label'])
for i in range(len(dev)):
    if(dev['class_label'][i]==1):
        dict = {'topic':dev['topic'][i] , 'tweet_id':dev['tweet_id'][i], 'tweet_text':aug.augment(dev['tweet_text'][i]), 'class_label':dev['class_label'][i] }
        df2 = df2.append(dict, ignore_index = True)

In [11]:
df2.to_csv('augmented_data2.csv')

In [12]:
df.to_csv('augmented_data.csv')